# Loading data to Supabase Posgres Database

We have decided to try and host the database using Supabase as the backend. Hosting this on a Supabase's cloud, we could more easily scale and build applications that can access that database. We will eventually try this with the isochrone project.

In [1]:
import pandas as pd
import os
from supabase import create_client, Client

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Supabase API

We have created a database on Supabase off-code. This was created using my(nabilersyad) account under that organization. The database mirrors the SQLite database.
Details on the created database below.

SUPABASE_URL = 'saved locally in the shell. Get the details from supabase dashboard'  
SUPABASE_KEY = 'saved in the shell. Get the details from supabase dashboard'

database: transit-stations-database  
tables: 1. stations  
        2. entrances  
        3. station_entrances  

In [2]:
#declaring the supabase client we will working with
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)

In [3]:
#signin a user
#sign_in_response = supabase.auth.sign_in_with_password({"email": "nabile95@hotmail.com", "password": ""})


In [4]:

#user = supabase.auth.get_user()


## Stations table

Combining the various CSVs of different cities into one table and we will use this table to update data in the supabase database. Assume all data in this combined table to be the most updated

In [5]:

# Define the directory of cleansed data to transfer to sql
cleansed_data_directory = '../data_cleansed'
cleansed_kl_file = 'klang_valley_stations_cleansed.csv'
cleansed_montreal_file = 'montreal_stations_cleansed.csv'
cleansed_singapore_file = 'singapore_stations_cleansed.csv'
cleansed_combined_file = 'combined_stations_cleansed.csv'

# read cleaned dataframes
kl_data = pd.read_csv(os.path.join(cleansed_data_directory, cleansed_kl_file))
montreal_data = pd.read_csv(os.path.join(cleansed_data_directory, cleansed_montreal_file))
singapore_data = pd.read_csv(os.path.join(cleansed_data_directory, cleansed_singapore_file))


# Combine all the dataframes
stations_data_local = pd.concat([kl_data, montreal_data, singapore_data], axis=0, ignore_index=True)

#ensure NAs are in a posgres readable format
stations_data_local = stations_data_local.where(pd.notna(stations_data_local), None)

stations_data_local.index.name = 'station_id'
stations_data_local.to_csv(os.path.join(cleansed_data_directory, cleansed_combined_file), index=True)


# Print out the combined dataframe
stations_data_local

,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened
station_id,,,,,,,,,,,,,,
0,KL Sentral,KA01,Keretapi Tanah Melayu,3.134603,101.686567,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
1,Kuala Lumpur,KA02,Keretapi Tanah Melayu,3.139513,101.693789,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
2,Bank Negara,KA03,Keretapi Tanah Melayu,3.154542,101.693010,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
3,Putra,KA04,Keretapi Tanah Melayu,3.165005,101.691234,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
4,Mid Valley,KB01,Keretapi Tanah Melayu,3.118528,101.678985,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,Outram Park MRT Station,TE17,None,1.280400,103.840100,TEL,Thomson-East Coast Line,None,Brown,#734538,Singapore,None,None,None
532,Maxwell MRT Station,TE18,None,1.280600,103.844000,TEL,Thomson-East Coast Line,None,Brown,#734538,Singapore,None,None,None
533,Shenton Way MRT Station,TE19,None,1.277540,103.850770,TEL,Thomson-East Coast Line,None,Brown,#734538,Singapore,None,None,None


We will now retrieve the equivalent table in supabase. This will be the table we need to update

In [6]:
response = supabase.table('stations').select("*").execute()
data,_ = response
stations_data_supa = pd.DataFrame(data[1])
stations_data_supa.set_index('station_id',inplace=True)
stations_data_supa

,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened
station_id,,,,,,,,,,,,,,
1,Kuala Lumpur,KA02,Keretapi Tanah Melayu,3.139513,101.693789,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
2,Bank Negara,KA03,Keretapi Tanah Melayu,3.154542,101.693010,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
3,Putra,KA04,Keretapi Tanah Melayu,3.165005,101.691234,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
4,Mid Valley,KB01,Keretapi Tanah Melayu,3.118528,101.678985,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
5,Seputeh,KB02,Keretapi Tanah Melayu,3.113697,101.681299,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,Berri–Uqam,G15,None,45.514946,-73.561340,None,Green Line,None,Green,#5F8C55,Montreal,Rue Berri |Université du Québec à Montréal,"Simon Després dit Le Berry, land owner (1659) ...",14-10-1966
279,Pie-Ix,G21,None,45.554200,-73.551500,None,Green Line,None,Green,#5F8C55,Montreal,Boulevard Pie-IX,Pope Pius IX,06-06-1976
300,Square-Victoria–Oaci,O15,None,45.502033,-73.563120,None,Orange Line,None,Orange,#D06D07,Montreal,Square Victoria |International Civil Aviation ...,Queen Victoria |Nearby headquarters of ICAO,06-02-1967


In [7]:
stations_data_supa.loc[264]

name                                                               Lasalle
station_code                                                            G6
service_provider_name                                                 None
latitude                                                         45.470906
longitude                                                        -73.56614
route_id                                                              None
route_name                                                      Green Line
line_number                                                           None
line_colour                                                          Green
colour_hex_code                                                    #5F8C55
region                                                            Montreal
odonym                                                   Boulevard LaSalle
namesake                 Robert Cavelier de La Salle, French explorer, ...
opened                   

### Merging local and supabase tables (Stations)

We will merge the two databases on their index in a union merge while creating  a new column to indicate for each row, which table has it

There will be 4 cases

Case 1: Row is in both local and supa table and is identical

Case 2: Row is in both tables but has some differences in one or more columns

Case 3: rows in local but not in supa - only_local

Case 4: rows not in local but in supa - only_supa


In [8]:
# Perform an outer join on the dataframes
merged_stations_data = pd.merge(stations_data_local, stations_data_supa, how='outer',left_index=True,right_index=True,indicator=True)

# Case 1 and 2: rows exist in both dataframes
both_stations_data = merged_stations_data[merged_stations_data['_merge'] == 'both']
both_stations_data = both_stations_data.drop(columns=['_merge'])

merged_stations_data['_merge'].value_counts()



both          536
left_only       0
right_only      0
Name: _merge, dtype: int64

#### Case 1 handling
We will identify the rows that are in both tables and identical from the both_data table using the 'both' indicator.

This is the extend of work that needs to be done. The supabase table does not need to be updated

In [9]:
#Case 1 
identical_stations_rows_mask = [stations_data_local.loc[i, stations_data_local.columns].equals(stations_data_supa.loc[i, stations_data_supa.columns])
                        for i in both_stations_data.index]
identical_stations_rows = both_stations_data[identical_stations_rows_mask]
identical_stations_rows = stations_data_local.loc[identical_stations_rows.index]
identical_stations_rows

,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened
station_id,,,,,,,,,,,,,,
0,KL Sentral,KA01,Keretapi Tanah Melayu,3.134603,101.686567,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
1,Kuala Lumpur,KA02,Keretapi Tanah Melayu,3.139513,101.693789,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
2,Bank Negara,KA03,Keretapi Tanah Melayu,3.154542,101.693010,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
3,Putra,KA04,Keretapi Tanah Melayu,3.165005,101.691234,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
4,Mid Valley,KB01,Keretapi Tanah Melayu,3.118528,101.678985,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,Outram Park MRT Station,TE17,None,1.280400,103.840100,TEL,Thomson-East Coast Line,None,Brown,#734538,Singapore,None,None,None
532,Maxwell MRT Station,TE18,None,1.280600,103.844000,TEL,Thomson-East Coast Line,None,Brown,#734538,Singapore,None,None,None
533,Shenton Way MRT Station,TE19,None,1.277540,103.850770,TEL,Thomson-East Coast Line,None,Brown,#734538,Singapore,None,None,None


#### Case 2 handling
Using the mask from handling case 1, we can get the rows with different values by getting the inverse of the identical_rows_mask to get right index. Then using this index to select the relevant rows from both_data

We will then use the rows from different_rows to update the supabase table using the update() function

In [10]:
different_stations_rows_mask= [not b for b in identical_stations_rows_mask]
different_stations_rows = both_stations_data[different_stations_rows_mask]

# Replace all NaN values in the DataFrame with None
different_stations_rows = different_stations_rows.where(pd.notna(different_stations_rows), None)
different_stations_rows = stations_data_local.loc[different_stations_rows.index]

different_stations_rows

,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened
station_id,,,,,,,,,,,,,,


In [11]:
#the following is to convert index in pandas to be a column, usable for posgres
different_stations_rows = different_stations_rows.reset_index(drop=False)

# For the different rows, you want to update data_supabase with data from data_local
for index, row in different_stations_rows.iterrows():
    # Use Supabase update method
    # Note: replace 'id' and 'your_table' with your actual id column name and table name

    data, error = supabase.table('stations').update(row.to_dict()).eq('station_id', row['station_id']).execute()


#### Case 3 handling
Using the left_only indicator, we can get the rows that are in local that needs to be inserted in the supabase table

We will then use the rows from only_local to update the supabase table using the insert() function

In [12]:

# Case 3: rows in data_local but not in data_supabase

only_local_stations = merged_stations_data[merged_stations_data['_merge'] == 'left_only']
only_local_stations = only_local_stations.drop(columns=['_merge'])

only_local_stations = stations_data_local.loc[only_local_stations.index]
only_local_stations = only_local_stations.where(pd.notna(only_local_stations), None)
only_local_stations = only_local_stations.reset_index(drop=False)

only_local_stations

,station_id,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened


In [17]:
# For these rows, you want to insert into data_supabase

 # Check if DataFrame is empty
if not only_local_stations.empty:
    data, error = supabase.table('stations').insert(only_local_stations.to_dict('records')).execute()
else:
    print("No new rows to be inserted")



No new rows to be inserted


#### Case 4 handling
Using the right_only indicator, we can get the rows that are in supabase that needs to be removed from the supabase table

We will then use the rows from only_local to update the supabase table using the delete() function

In [14]:
# Case 4: rows not in data_local but in data_supabase
only_supabase_stations = merged_stations_data[merged_stations_data['_merge'] == 'right_only']
only_supabase_stations = only_supabase_stations.drop(columns=['_merge'])

only_supabase_stations = stations_data_local.loc[only_supabase_stations.index]
only_supabase_stations = only_supabase_stations.where(pd.notna(only_supabase_stations), None)
only_supabase_stations = only_supabase_stations.reset_index(drop=False)
only_supabase_stations

,station_id,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened


In [15]:
# For these rows, you want to delete from data_supabase
for index, row in only_supabase_stations.iterrows():
    # Use Supabase delete method
    data, error = supabase.table('stations').delete().eq('station_id', row['station_id']).execute()

## Entrances tables

There are two entrances tables that we need to create. We'll call them entrances and station_entrances.
Again assume all local data to be the most updated

We will start with entrances table

In [16]:
# Retrieve entrances table in local csv file
cleansed_entrances = 'klang_valley_entrances_cleansed.csv'

# read cleaned entrances dataframes
entrances_data_local = pd.read_csv(os.path.join(cleansed_data_directory, cleansed_entrances))

#ensure NAs are in a posgres readable format
entrances_data_local = entrances_data_local.where(pd.notna(entrances_data_local), None)

#ensure index to be entrance_id because it is easier to work with
entrances_data_local.set_index('entrance_id',inplace=True)
entrances_data_local = entrances_data_local.sort_values(by='entrance_id')

entrances_data_local

,longitude,latitude,entrance_destination,entrance_name
entrance_id,,,,
1544031348,101.711374,3.145929,None,B
1631412559,101.604933,3.113208,None,None
1632120095,101.694340,3.142320,None,E
2278515570,101.644077,3.050650,None,None
2686635178,101.699182,3.138565,None,C
...,...,...,...,...
11871659780,101.693472,3.140096,None,A
11899741782,101.710384,3.145855,None,B1
11925140813,101.686758,3.133750,None,None


In [17]:
# Retrieve entrances table from supabase

response = supabase.table('entrances').select("*").execute()
data,_ = response
entrances_data_supa = pd.DataFrame(data[1])

#ensure index to be entrance_id because it is easier to work with
entrances_data_supa.set_index('entrance_id',inplace=True)
entrances_data_supa = entrances_data_supa.sort_values(by='entrance_id')

entrances_data_supa

,longitude,latitude,entrance_destination,entrance_name
entrance_id,,,,
1544031348,101.711374,3.145929,None,B
1631412559,101.604933,3.113208,None,None
1632120095,101.694340,3.142320,None,E
2278515570,101.644077,3.050650,None,None
2686635178,101.699182,3.138565,None,C
...,...,...,...,...
11768421750,101.693051,3.154850,None,None
11871659779,101.693291,3.139477,None,B
11871659780,101.693472,3.140096,None,A


### Merging local and supabase tables

We will merge the two databases on their index in a union merge while creating  a new column to indicate for each row, which table has it

There will be 4 cases

Case 1: Row is in both local and supa table and is identical

Case 2: Row is in both tables but has some differences in one or more columns

Case 3: rows in local but not in supa - only_local

Case 4: rows not in local but in supa - only_supa

In [18]:
# Perform an outer join on the dataframes
merged_data = pd.merge(entrances_data_local,entrances_data_supa,how='outer',left_index=True,right_index=True,indicator=True)

# Case 1 and 2: rows exist in both dataframes
both_data = merged_data[merged_data['_merge'] == 'both']
both_data = both_data.drop(columns=['_merge'])

merged_data['_merge'].value_counts()


both          275
left_only       3
right_only      0
Name: _merge, dtype: int64

#### Case 1 handling
We will identify the rows that are in both tables and identical from the both_data table using the 'both' indicator.

This is the extend of work that needs to be done. The supabase table does not need to be updated

In [19]:
#Case 1 
identical_rows_mask = [entrances_data_local.loc[i, entrances_data_local.columns].equals(entrances_data_supa.loc[i, entrances_data_supa.columns])
                        for i in both_data.index]
identical_rows = both_data[identical_rows_mask]
identical_rows = entrances_data_local.loc[identical_rows.index]
identical_rows

,longitude,latitude,entrance_destination,entrance_name
entrance_id,,,,
1544031348,101.711374,3.145929,None,B
1631412559,101.604933,3.113208,None,None
1632120095,101.694340,3.142320,None,E
2278515570,101.644077,3.050650,None,None
2686635178,101.699182,3.138565,None,C
...,...,...,...,...
11768421750,101.693051,3.154850,None,None
11871659779,101.693291,3.139477,None,B
11871659780,101.693472,3.140096,None,A


#### Case 2 handling
Using the mask from handling case 1, we can get the rows with different values by getting the inverse of the identical_rows_mask to get right index. Then using this index to select the relevant rows from both_data

We will then use the rows from different_rows to update the supabase table using the update() function

In [20]:
different_rows_mask= [not b for b in identical_rows_mask]
different_rows = both_data[different_rows_mask]

# Replace all NaN values in the DataFrame with None
different_rows = different_rows.where(pd.notna(different_rows), None)
different_rows = entrances_data_local.loc[different_rows.index]

different_rows

,longitude,latitude,entrance_destination,entrance_name
entrance_id,,,,


In [21]:
#the following is to convert index in pandas to be a column, usable for posgres
different_rows = different_rows.reset_index(drop=False)

# For the different rows, you want to update data_supabase with data from data_local
for index, row in different_rows.iterrows():
    # Use Supabase update method
    # Note: replace 'id' and 'your_table' with your actual id column name and table name
    data, error = supabase.table('entrances').update(row.to_dict()).eq('entrance_id', row['entrance_id']).execute()


#### Case 3 handling
Using the left_only indicator, we can get the rows that are in local that needs to be inserted in the supabase table

We will then use the rows from only_local to update the supabase table using the insert() function

In [22]:

# Case 3: rows in data_local but not in data_supabase

only_local = merged_data[merged_data['_merge'] == 'left_only']
only_local = only_local.drop(columns=['_merge'])

only_local = entrances_data_local.loc[only_local.index]
only_local = only_local.where(pd.notna(only_local), None)
only_local = only_local.reset_index(drop=False)

only_local

,entrance_id,longitude,latitude,entrance_destination,entrance_name
0,2688004520,101.716143,3.128749,None,None
1,11991808923,101.678695,3.119295,None,None
2,12037033085,101.715076,3.128402,None,None


In [23]:
# For these rows, you want to insert into data_supabase
if not only_local.empty:
    data, error = supabase.table('entrances').insert(only_local.to_dict('records')).execute()   
else:
        print("No new rows to be inserted")




#### Case 4 handling
Using the right_only indicator, we can get the rows that are in supabase that needs to be removed from the supabase table

We will then use the rows from only_local to update the supabase table using the delete() function

In [24]:
# Case 4: rows not in data_local but in data_supabase
only_supabase = merged_data[merged_data['_merge'] == 'right_only']
only_supabase = only_supabase.drop(columns=['_merge'])

only_supabase = entrances_data_supa.loc[only_supabase.index]
only_supabase = only_supabase.where(pd.notna(only_supabase), None)
only_supabase = only_supabase.reset_index(drop=False)
only_supabase

,entrance_id,longitude,latitude,entrance_destination,entrance_name


In [25]:
# For these rows, you want to delete from data_supabase
for index, row in only_supabase.iterrows():
    # Use Supabase delete method
    data, error = supabase.table('entrances').delete().eq('entrance_id', row['entrance_id']).execute()

## Station Entrances tables

This table tracks the relationship between station and entrances

In [26]:
# Retrieve station entrances table in local csv file
cleansed_station_entrances= 'klang_valley_stations_entrances_relation_cleansed.csv'

# read cleaned statuion entrances dataframes
station_entrances_data_local = pd.read_csv(os.path.join(cleansed_data_directory, cleansed_station_entrances))

#ensure NAs are in a posgres readable format
station_entrances_data_local = station_entrances_data_local.where(pd.notna(station_entrances_data_local), None)

#ensure index to be entrance_id because it is easier to work with
station_entrances_data_local.set_index('relationship_id',inplace=True)
station_entrances_data_local = station_entrances_data_local.sort_values(by='entrance_id')

station_entrances_data_local


,entrance_id,station_name,station_code
relationship_id,,,
169,1544031348,Bukit Bintang (Monorail),MR6
123,1631412559,Kelana Jaya,KJ24
114,2278515570,Kinrara BK 5,SP22
219,2686635178,Maharajalela,MR3
43,3308608988,KLCC,KJ10
...,...,...,...
230,11768421750,KTM Bank Negara,KA03
226,11871659779,Kuala Lumpur,Unnamed
225,11871659780,Kuala Lumpur,Unnamed


In [27]:
# Retrieve entrances table from supabase

response = supabase.table('station_entrances').select("*").execute()
data,_ = response
station_entrances_data_supa = pd.DataFrame(data[1])

#ensure index to be entrance_id because it is easier to work with
station_entrances_data_supa.set_index('relationship_id',inplace=True)
station_entrances_data_supa = station_entrances_data_supa.sort_values(by='relationship_id')

station_entrances_data_supa

,entrance_id,station_name,station_code
relationship_id,,,
0,10796851698,Pudu,AG10
1,10796851698,Pudu,SP10
2,5485710279,Kampung Baru,KJ11
3,5485710278,Kampung Baru,KJ11
4,9740843587,Masjid Jamek (KJ),KJ13
...,...,...,...
304,5044809586,Tun Razak Exchange (PY),PY23
305,10658294223,Cyberjaya City Centre,PY30
306,10722980582,Putrajaya Sentral (MRT),PY41


In [28]:
station_entrances_data_local[station_entrances_data_local['station_code']=='KC02']

,entrance_id,station_name,station_code
relationship_id,,,
236,11264561181,Batu Kentonmen,KC02


In [29]:
station_entrances_data_supa[station_entrances_data_supa['station_code']=='KC02']

,entrance_id,station_name,station_code
relationship_id,,,


### Merging local and supabase tables

We will merge the two databases on their index in a union merge while creating  a new column to indicate for each row, which table has it

There will be 4 cases

Case 1: Row is in both local and supa table and is identical

Case 2: Row is in both tables but has some differences in one or more columns

Case 3: rows in local but not in supa - only_local

Case 4: rows not in local but in supa - only_supa

In [30]:
# Perform an outer join on the dataframes
merged_station_entrances_data = pd.merge(station_entrances_data_local,station_entrances_data_supa,how='outer',left_index=True,right_index=True,indicator=True)

# Case 1 and 2: rows exist in both dataframes
both_station_entrances_data = merged_station_entrances_data[merged_station_entrances_data['_merge'] == 'both']
both_station_entrances_data = both_station_entrances_data.drop(columns=['_merge'])

merged_station_entrances_data['_merge'].value_counts()


both          309
left_only       1
right_only      0
Name: _merge, dtype: int64

#### Case 1 handling
We will identify the rows that are in both tables and identical from the both_data table using the 'both' indicator.

This is the extend of work that needs to be done. The supabase table does not need to be updated

In [31]:
#Case 1 
identical_station_entrances_rows_mask = [station_entrances_data_local.loc[i, station_entrances_data_local.columns].equals(station_entrances_data_supa.loc[i, station_entrances_data_supa.columns])
                        for i in both_station_entrances_data.index]
identical_station_entrances_rows = both_station_entrances_data[identical_station_entrances_rows_mask]
identical_station_entrances_rows = station_entrances_data_local.loc[identical_station_entrances_rows.index]
identical_station_entrances_rows

,entrance_id,station_name,station_code
relationship_id,,,


#### Case 2 handling
Using the mask from handling case 1, we can get the rows with different values by getting the inverse of the identical_rows_mask to get right index. Then using this index to select the relevant rows from both_data

We will then use the rows from different_rows to update the supabase table using the update() function

In [32]:
different_station_entrances_rows_mask= [not b for b in identical_station_entrances_rows_mask]
different_station_entrances_rows = both_station_entrances_data[different_station_entrances_rows_mask]

# Replace all NaN values in the DataFrame with None
different_station_entrances_rows = different_station_entrances_rows.where(pd.notna(different_station_entrances_rows), None)
different_station_entrances_rows = station_entrances_data_local.loc[different_station_entrances_rows.index]

different_station_entrances_rows

,entrance_id,station_name,station_code
relationship_id,,,
0,11435336038,Plaza Rakyat,AG8
1,11435336038,Plaza Rakyat,SP8
2,10796851698,Pudu,AG10
3,10796851698,Pudu,SP10
4,5485710279,Kampung Baru,KJ11
...,...,...,...
304,10611678882,16 Sierra,PY38
305,10839997852,Cyberjaya Utara,PY39
306,10658294223,Cyberjaya City Centre,PY30


#### Case 3 handling
Using the left_only indicator, we can get the rows that are in local that needs to be inserted in the supabase table

We will then use the rows from only_local to update the supabase table using the insert() function

In [33]:

# Case 3: rows in data_local but not in data_supabase

only_local_station_entrances = merged_station_entrances_data[merged_station_entrances_data['_merge'] == 'left_only']
only_local_station_entrances = only_local_station_entrances.drop(columns=['_merge'])

only_local_station_entrances = station_entrances_data_local.loc[only_local_station_entrances.index]
only_local_station_entrances = only_local_station_entrances.where(pd.notna(only_local_station_entrances), None)
only_local_station_entrances = only_local_station_entrances.reset_index(drop=False)

only_local_station_entrances

,relationship_id,entrance_id,station_name,station_code
0,309,5044809586,Tun Razak Exchange (PY),PY23


In [34]:
# For these rows, you want to insert into data_supabase
if not only_local_station_entrances.empty:
    data, error = supabase.table('station_entrances').insert(only_local_station_entrances.to_dict('records')).execute()
else:
    print("No new rows to be inserted")

#### Case 4 handling
Using the right_only indicator, we can get the rows that are in supabase that needs to be removed from the supabase table

We will then use the rows from only_local to update the supabase table using the delete() function

In [35]:
# Case 4: rows not in data_local but in data_supabase
only_supabase_station_entrances = merged_station_entrances_data[merged_station_entrances_data['_merge'] == 'right_only']
only_supabase_station_entrances = only_supabase_station_entrances.drop(columns=['_merge'])

only_supabase_station_entrances = station_entrances_data_local.loc[only_supabase_station_entrances.index]
only_supabase_station_entrances = only_supabase_station_entrances.where(pd.notna(only_supabase_station_entrances), None)
only_supabase_station_entrances = only_supabase_station_entrances.reset_index(drop=False)
only_supabase_station_entrances

,relationship_id,entrance_id,station_name,station_code


In [36]:
# For these rows, you want to delete from data_supabase
for index, row in only_supabase_station_entrances.iterrows():
    # Use Supabase delete method
    data, error = supabase.table('station_entrances').delete().eq('relationship_id', row['relationship_id']).execute()